In [1]:
%load_ext autoreload
%autoreload 2  

In [2]:
#!/usr/bin/env python3
"""

Action Stream Inference Demo - Environment Setup
"""

import os
import sys
import argparse
from pathlib import Path
import warnings
import functools
import types
from typing import Union, Optional, List, Dict, Any
warnings.filterwarnings('ignore')

# Add the parent directories to Python path for imports
current_dir = os.getcwd()
parent_dir = os.path.join(current_dir, "..", "..")
sys.path.append(parent_dir)
print(f"Added to Python path: {parent_dir}")

# Core imports
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from trl.trainer.dpo_trainer import DataCollatorForPreference
import numpy as np
import tqdm

# Append current directory so that interpreter can find experiments.robot
from experiments.robot.libero.libero_utils import (
    get_libero_dummy_action,
    get_libero_env,
    get_libero_image,
    quat2axisangle,
    save_rollout_video_CoA,
)
from experiments.robot.robot_utils import (
    get_CoA,
)

# Add the parent directories to Python path for imports
current_dir = os.getcwd()
parent_dir = os.path.join(current_dir, "..", "..")
sys.path.append(parent_dir)
print(f"Added to Python path: {parent_dir}")
# Local imports
try:
    from infer_utils import predict_CoA, add_text_to_image
    from src.config import GenerateConfig
    from src.model_utils import setup_vla_model_with_lora,  setup_logging_and_environment
    print("✓ Successfully imported local modules")
except ImportError as e:
    print(f"✗ Failed to import local modules: {e}")
    print("Please ensure you're running from the correct directory")


# Check GPU availability
print(f"\nGPU Information:")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.1f} GB")

print("\n" + "="*50)
print("Environment setup completed!")
print("="*50)


Added to Python path: /mnt/sda/home/zijianwang/openvla/vla-scripts/DPO/../..


2025-08-31 19:57:40.341449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-31 19:57:40.341483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-31 19:57:40.343037: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-31 19:57:40.352039: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-31 19:57:41.203407: W tensorflow/comp

Added to Python path: /mnt/sda/home/zijianwang/openvla/vla-scripts/DPO/../..
✓ Successfully imported local modules

GPU Information:
CUDA available: True
GPU count: 4
  GPU 0: NVIDIA GeForce RTX 4090
    Memory: 25.4 GB
  GPU 1: NVIDIA GeForce RTX 4090
    Memory: 25.4 GB
  GPU 2: NVIDIA GeForce RTX 4090
    Memory: 25.4 GB
  GPU 3: NVIDIA RTX A6000
    Memory: 51.0 GB

Environment setup completed!


In [3]:
"""
Configuration Setup - Modify parameters here for different experiments
"""

# ====== TRAINING PARAMETERS ======
DEVICE = "cuda:0"  # Device for policy model
STREAM_LENGTH = 10        # Stream length for trajectory processing

# ====== WANDB CONFIGURATION ======
USE_WANDB = False         # Set to True to enable Weights & Biases logging
WANDB_PROJECT = "openvla_CoA_DPO_demo"
WANDB_ENTITY = "15652388600"
RUN_ID_NOTE = "notebook_demo"

# ====== PATH CONFIGURATION ======
ROOT_DIR = "/mnt/sda/home/zijianwang"


# Optional: Override default paths (leave empty to use defaults)
PRETRAINED_CHECKPOINT = f"{ROOT_DIR}/openvla/FT_res/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25"
LORA_PATH = f"{ROOT_DIR}/openvla/adapter_tmp_dir/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25"
BASE_VLA_PATH = f"{ROOT_DIR}/HF_CACHE/openvla-7b-finetuned-libero-10"


WINNER_TRAJECTORY_PATH = f"{ROOT_DIR}/openvla/vla-scripts/DPO/winner_trajectory"
ADAPTER_TMP_DIR = f"{ROOT_DIR}/openvla/DPO_adapter_tmp_dir"
RUN_ROOT_DIR = f"{ROOT_DIR}/openvla/DPO_res"

TASK_SUITE_NAME = "libero_10"
TASK_NUM = 1            # Set to specific task number or None for all tasks
# Create configuration objects
print("Creating configuration...")

# Policy model configuration
model_cfg = GenerateConfig(
    root_dir=ROOT_DIR,
    device=DEVICE,
    stream_length=STREAM_LENGTH,
    use_wandb=USE_WANDB,
    wandb_project=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY,
    run_id_note=RUN_ID_NOTE,
    grad_accumulation_steps=1,
    pretrained_checkpoint=PRETRAINED_CHECKPOINT,
    lora_path=LORA_PATH,
    base_vla_path=BASE_VLA_PATH,
    winner_trajectory_path=WINNER_TRAJECTORY_PATH,
    adapter_tmp_dir=ADAPTER_TMP_DIR,
    run_root_dir=RUN_ROOT_DIR,
    task_num=TASK_NUM
)

print("\n" + "="*50)
print("CONFIGURATION SUMMARY")
print("="*50)
print(f"Policy Device: {model_cfg.device}")
print(f"Stream Length: {model_cfg.stream_length}")
print(f"Task Number: {model_cfg.task_num if model_cfg.task_num else 'All tasks'}")
print("\nPath Configuration:")
print(f"Root Dir: {model_cfg.root_dir}")
print(f"Pretrained Checkpoint: {model_cfg.pretrained_checkpoint}")
print(f"LoRA Path: {model_cfg.lora_path}")
print(f"Winner Trajectory Path: {model_cfg.winner_trajectory_path}")
print(f"Adapter Tmp Dir: {model_cfg.adapter_tmp_dir}")
print(f"Run Root Dir: {model_cfg.run_root_dir}")
print("="*50)


Creating configuration...

CONFIGURATION SUMMARY
Policy Device: cuda:0
Stream Length: 10
Task Number: 1

Path Configuration:
Root Dir: /mnt/sda/home/zijianwang
Pretrained Checkpoint: /mnt/sda/home/zijianwang/openvla/FT_res/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25
LoRA Path: /mnt/sda/home/zijianwang/openvla/adapter_tmp_dir/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25
Winner Trajectory Path: /mnt/sda/home/zijianwang/openvla/vla-scripts/DPO/winner_trajectory
Adapter Tmp Dir: /mnt/sda/home/zijianwang/openvla/DPO_adapter_tmp_dir
Run Root Dir: /mnt/sda/home/zijianwang/openvla/DPO_res


In [4]:
"""
Model Loading Section
"""

print("Starting model loading...")
print("This may take several minutes depending on model size and device speed.")
print("\n" + "-"*30)

# Load policy model with LoRA
print("[1/2] Loading policy model (with LoRA)...")
print(f"Target device: {model_cfg.device}")

try:
    policy_model = setup_vla_model_with_lora(model_cfg)
    print(f"✓ Policy model loaded successfully")
    print(f"Model device: {next(policy_model.parameters()).device}")
    print(f"Model dtype: {next(policy_model.parameters()).dtype}")
    
    # Count parameters
    total_params = sum(p.numel() for p in policy_model.parameters())
    trainable_params = sum(p.numel() for p in policy_model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Trainable ratio: {100 * trainable_params / total_params:.2f}%")
    
except Exception as e:
    print(f"✗ Failed to load policy model: {e}")
    raise


# Add CoA prediction method to model
top_k = 1
# 创建一个带默认top_k=2的函数
predict_CoA_with_defaults = functools.partial(predict_CoA, top_k=top_k)

# 添加到model
policy_model.get_CoA = types.MethodType(predict_CoA_with_defaults, policy_model) 

print("[*] Setting up logging and environment...")
processor, log_file, task_suite, num_tasks_in_suite, resize_size = setup_logging_and_environment(model_cfg, policy_model)

Starting model loading...
This may take several minutes depending on model size and device speed.

------------------------------
[1/2] Loading policy model (with LoRA)...
Target device: cuda:0
[*] Instantiating Pretrained VLA model
[*] Loading in BF16 with Flash-Attention Enabled


08/31 [19:58:09] WARNING  | >> Expected `transformers==4.40.1` and `tokenizers==0.19.1`   ]8;id=75954;file:///home/zijianwang/.cache/huggingface/modules/transformers_modules/openvla/openvla-7b/31f090d05236101ebfc381b61c674dd4746d4ce0/modeling_prismatic.py\modeling_prismatic.py]8;;\:]8;id=861168;file:///home/zijianwang/.cache/huggingface/modules/transformers_modules/openvla/openvla-7b/31f090d05236101ebfc381b61c674dd4746d4ce0/modeling_prismatic.py#228\228]8;;\
                          but got `transformers==4.53.3` and `tokenizers==0.21.2`; there                           
                          might be inference-time regressions due to dependency changes.                           
                          If in doubt, pleaseuse the above versions.                                               

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 10.56it/s]


✓ Policy model loaded successfully
Model device: cuda:0
Model dtype: torch.bfloat16
Total parameters: 7,707,479,616
Trainable parameters: 166,242,432
Trainable ratio: 2.16%
[*] Setting up logging and environment...
Logging to local log file: ./experiments/logs/DPO-libero_10-openvla-2025_08_31-19_58_02--notebook_demo.txt
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task suite: libero_10


In [5]:
def get_max_steps(task_suite_name: str) -> int:
    """Get maximum steps for different task suites."""
    max_steps_dict = {
        "libero_spatial": 220,  # longest training demo has 193 steps
        "libero_object": 280,   # longest training demo has 254 steps
        "libero_goal": 300,     # longest training demo has 270 steps
        "libero_10": 550,       # longest training demo has 505 steps
        "libero_90": 400,       # longest training demo has 373 steps
    }
    return max_steps_dict.get(task_suite_name, 300)

In [6]:
print(range(num_tasks_in_suite), model_cfg.num_trials_per_task)

range(0, 10) 50


In [7]:
# Run evaluation
human_prompt_template = "What sequence of actions should the robot take to {lang}?"

print("[*] Starting evaluation...")
"""Run the main evaluation loop."""
# Start evaluation
total_episodes, total_successes = 0, 0
total_num_act_units = 40
max_steps = get_max_steps(model_cfg.task_suite_name)
for task_id in tqdm.tqdm(range(num_tasks_in_suite)):
    if task_id != model_cfg.task_num: 
        continue
    # Get task
    task = task_suite.get_task(task_id)
    
    # Get default LIBERO initial states
    initial_states = task_suite.get_task_init_states(task_id)

    # Initialize LIBERO environment and task description
    env, task_description = get_libero_env(task, model_cfg.model_family, resolution=256)

    # Start episodes
    task_episodes, task_successes = 0, 0
    for episode_idx in tqdm.tqdm(range(model_cfg.num_trials_per_task)):
        # if episode_idx < 6:  # Currently only running first episode
        #     continue
        print(f"\nTask: {task_description}")
        log_file.write(f"\nTask: {task_description}\n")

        # Set initial states
        obs = env.set_init_state(initial_states[episode_idx])

        print(f"Starting episode {episode_idx}...")
        log_file.write(f"Starting episode {episode_idx}...\n")
        num_act_episode, num_act_successes = 0, 0
        for num_act_units in range(1, total_num_act_units):  # Try from 1 to 100 units
            if num_act_units != 10:
                continue
            print(f"num_act_units: {num_act_units}")
            log_file.write(f"num_act_units: {num_act_units}\n")
            total_length = 0
            replay_images = []
            env.reset()
            obs = env.set_init_state(initial_states[episode_idx])
            CoA_step = 0
            while total_length < max_steps + model_cfg.num_steps_wait:
                try:
                    # IMPORTANT: Do nothing for the first few timesteps because the simulator drops objects
                    # and we need to wait for them to fall
                    if total_length < model_cfg.num_steps_wait:
                        obs, reward, done, info = env.step(get_libero_dummy_action(model_cfg.model_family))
                        total_length += 1
                        continue

                    # Get preprocessed image
                    img = get_libero_image(obs, resize_size) #np.array [224, 224, 3]
                    # Prepare observations dict
                    # Note: OpenVLA does not take proprio state as input
                    observation = {
                        "full_image": img,
                        "state": np.concatenate(
                            (obs["robot0_eef_pos"], quat2axisangle(obs["robot0_eef_quat"]), obs["robot0_gripper_qpos"])
                        ),
                        }
                    CoA:List[np.ndarray] = get_CoA(
                        model_cfg,
                        policy_model,
                        observation,
                        task_description,
                        processor=processor,
                        num_act_units=num_act_units,
                        human_prompt_template=human_prompt_template
                    ) 
                    CoA_step += 1   #[array([0.002, -0.002, 0.000, -0.000, -0.000, -0.001, -1.000])]
                    current_CoA_length = len(CoA)
                    total_length = total_length + current_CoA_length
                    print(f"total_length:{total_length}/{max_steps}", end="\r")
                    for unit_idx, action in enumerate(CoA):
                        obs, reward, done, info = env.step(action.tolist())
                        temp_img = get_libero_image(obs, resize_size) #temp_img: np.ndarray, shape: (224, 224, 3)
                        temp_img = add_text_to_image(temp_img, num_act_units, CoA_step)
                        replay_images.append(temp_img)
                    if done:
                        print(f"Success at task {task_id} -- episode {episode_idx} -- num_act_units:{num_act_units}, using t = {total_length}")
                        log_file.write(f"Success at task {task_id} -- episode {episode_idx} -- num_act_units:{num_act_units}, using t = {total_length}\n")
                        task_successes += 1
                        num_act_successes += 1
                        total_successes += 1    
                        break
                except Exception as e:
                    print(f"Caught exception: {e}")
                    log_file.write(f"Caught exception: {e}\n")
                    break
            task_episodes += 1
            num_act_episode += 1
            total_episodes += 1
            save_rollout_video_CoA(
                    replay_images, episode_idx, success=done, task_description=task_description, log_file=log_file, num_act_units=num_act_units, task_id = task_id, task_suite_name = model_cfg.task_suite_name, top_k=top_k
                )
            # Log current results
            print(f"Success: {done}")
            print(f"# episodes completed so far: {total_episodes}")
            print(f"# successes: {total_successes} ({total_successes / total_episodes * 100:.1f}%)")
            log_file.write(f"Success: {done}\n")
            log_file.write(f"# episodes completed so far: {total_episodes}\n")
            log_file.write(f"# successes: {total_successes} ({total_successes / total_episodes * 100:.1f}%)\n")
            log_file.flush()

        # Log final results
        print(f"Current task success rate: {float(task_successes) / float(task_episodes)}")
        print(f"Current total success rate: {float(total_successes) / float(total_episodes)}")
        log_file.write(f"Current task success rate: {float(task_successes) / float(task_episodes)}\n")
        log_file.write(f"Current total success rate: {float(total_successes) / float(total_episodes)}\n")
        log_file.flush()
    # Save local log file
    # log_file.close()


[*] Starting evaluation...


  0%|          | 0/10 [00:00<?, ?it/s]

[Warning]: datasets path /mnt/sda/home/zijianwang/LIBERO/libero/libero/../datasets does not exist!
[Warning]: datasets path /mnt/sda/home/zijianwang/LIBERO/libero/libero/../datasets does not exist!



Task: put both the cream cheese box and the butter in the basket
Starting episode 0...
num_act_units: 10


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Saved rollout MP4 at path ./rollouts/libero_10/2025_08_31-19_58_02/task_id=1_put_both_the_cream_cheese_box_and_the_butter_in_th/epi=26--num_act=10--top_k=1--success=False.mp4
Success: False
# episodes completed so far: 27
# successes: 13 (48.1%)
Current task success rate: 0.48148148148148145
Current total success rate: 0.48148148148148145

Task: put both the cream cheese box and the butter in the basket
Starting episode 27...
num_act_units: 10
Success at task 1 -- episode 27 -- num_act_units:10, using t = 360


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saved rollout MP4 at path ./rollouts/libero_10/2025_08_31-19_58_02/task_id=1_put_both_the_cream_cheese_box_and_the_butter_in_th/epi=27--num_act=10--top_k=1--success=True.mp4
Success: True
# episodes completed so far: 28
# successes: 14 (50.0%)
Current task success rate: 0.5
Current total success rate: 0.5

Task: put both the cream cheese box and the butter in the basket
Starting episode 28...
num_act_units: 10


 10%|█         | 1/10 [1:48:50<16:19:37, 6530.79s/it]


KeyboardInterrupt: 